## EfficientNet 코드 구현

In [1]:
# EfficientNet-B0 Baseline network
## 1st = Conv 3*3
## 2nd = MBConv1, k3*3

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
from torchsummary import summary
from torch import optim
import torchvision
import torchvision.transforms as transforms

c:\pub\envs\pytorch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## SE Block

# squeeze and Excitation
# reduction_ratio = 0.25 (감소 비율)
# H X W X C -> 1 X 1 X C 로 펴준다음 다시 H X W X C로 바꿔주면서 각 채널마다 가중치 추가

class SEBlock(nn.Module):
    def __init__(self, in_channels, r=0.25):
        super().__init__()

        # C / r을 계산하는 변수
        # se_channels : reduce layer out channels 계산
        se_channels = max(1, int(in_channels*r))

        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # Global AvgPooling
            nn.Conv2d(in_channels, se_channels, kernel_size=1), # 1*1 conv
            nn.SiLU(),
            nn.Conv2d(se_channels, in_channels, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.se(x) # 가중치가 적용된 각각의 필터가 나옴

In [3]:
input = torch.rand(1, 3, 3)
print(input)
maxp = nn.AdaptiveAvgPool2d((1,1)) # 내가 원하는 크기의 pooling을 계산
output = maxp(input)
print(output)

tensor([[[0.1912, 0.1852, 0.4637],
         [0.9676, 0.9370, 0.0403],
         [0.0117, 0.3137, 0.5400]]])
tensor([[[0.4056]]])


In [4]:
# MBConv Block
## MobileNetv2 개념을 가져옴
# 1st Expantion
# 2nd Depth-wise Convolution
# 3rd Point-wise Convolution
# 4th Skip Connection

## 차이점
# LeRU6 대신 SiLU
# Depth-wise, Point-wise 사이 SE 사용
# Depth-wise 수행시 kernel_size를 3 or 5 사용

# 최종 모델
# 1st Expantion
# 2nd Depth-wise Convolution
# 2nd_1 : SEBlock
# 3rd Point-wise Convolution
# 4th Skip Connection

class MBConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expand, kernel_size, stride=1, r=0.25, dropout_rate=0.2, bias=True):
        super().__init__()

        # parameter 설정
        self.dropout_rate = dropout_rate
        self.expand = expand

        # skip connection 사용 조건
        self.use_residual = (in_channels == out_channels) and (stride == 1)

        # paper에서 수행한 BatchNorm, SiLU 적용
        # expand 채널을 늘리는 작업
        expand_channels = in_channels*expand
        self.expansion = nn.Sequential(nn.Conv2d(in_channels, expand_channels, 1, bias=False),
                                       nn.BatchNorm2d(expand_channels, momentum=0.99),
                                       nn.SiLU(),
                                       )
        # Depth-wise Convolution
        # groups = 1이면 모든 입력이 모든 출력과 conv 연산,
        # 각각의 input channel이 output channel과 대응 연산
        self.depth_wise = nn.Sequential(nn.Conv2d(expand_channels, expand_channels, kernel_size=kernel_size, stride=1, padding=1, groups=expand_channels),
                                        nn.BatchNorm2d(expand_channels, momentum=0.99),
                                        nn.SiLU(),
                                        )
        # Squeeze and Excitation
        self.se_block = SEBlock(expand_channels, r)

        # Point-wise Convolution
        self.point_wise = nn.Sequential(nn.Conv2d(expand_channels, out_channels, 1, 1, bias=False),
                                        nn.BatchNorm2d(out_channels, momentum=0.99),
                                        )
        
    def forward(self, x):
        if self.expand != 1:
            x = self.expansion(x)
        
        x = self.depth_wise(x)
        x = self.se_block(x)
        x = self.point_wise(x)

        res = x

        if self.use_residual:
            if self.training and (self.dropout_rate is not None):
                x = F.dropout2d(input=x, p=self.dropout_rate, training=self.training, inplace=True)
            
            x = x + res

        return x



In [5]:
class EfficientNet(nn.Module):
    def __init__(self, num_classes, width, depth, resolution, dropout):
        super().__init__()

        # stage 1
        out_ch = int(32 * width)
        self.stage1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=out_ch, kernel_size=3, stride=2, padding=1),
                                    nn.BatchNorm2d(out_ch, momentum=0.99),
                                    )
        # 다음 input 이미지가 반으로 줆,
        # stride가 2인 이유
        
        # stage 2
        self.stage2 = nn.Sequential(MBConvBlock(in_channels=out_ch, out_channels=16, expand=1, kernel_size=3, stride=1, dropout_rate=dropout))

        # stage 3
        self.stage3 = nn.Sequential(MBConvBlock(in_channels=16, out_channels=24, expand=6, kernel_size=3, stride=2, dropout_rate=dropout),
                                    MBConvBlock(in_channels=24, out_channels=24, expand=6, kernel_size=3, stride=1, dropout_rate=dropout),
                                    )
        
        # stage4
        self.stage4 = nn.Sequential(MBConvBlock(in_channels=24, out_channels=40, expand=6, kernel_size=5, stride=2, dropout_rate=dropout),
                                    MBConvBlock(in_channels=40, out_channels=40, expand=6, kernel_size=5, stride=1, dropout_rate=dropout),
                                   )
        
        # stage5
        self.stage5 = nn.Sequential(MBConvBlock(in_channels=40, out_channels=80, expand=6, kernel_size=3, stride=2, dropout_rate=dropout),
                                    MBConvBlock(in_channels=80, out_channels=80, expand=6, kernel_size=3, stride=1, dropout_rate=dropout),
                                    MBConvBlock(in_channels=80, out_channels=80, expand=6, kernel_size=3, stride=1, dropout_rate=dropout),
                                   )
        
        # stage6
        self.stage6 = nn.Sequential(MBConvBlock(in_channels=80, out_channels=112, expand=6, kernel_size=5, stride=1, dropout_rate=dropout),
                                    MBConvBlock(in_channels=112, out_channels=112, expand=6, kernel_size=5, stride=1, dropout_rate=dropout),
                                    MBConvBlock(in_channels=112, out_channels=112, expand=6, kernel_size=5, stride=1, dropout_rate=dropout),
                                   )
        
        # stage7
        self.stage7 = nn.Sequential(MBConvBlock(in_channels=112, out_channels=192, expand=6, kernel_size=5, stride=2, dropout_rate=dropout),
                                    MBConvBlock(in_channels=192, out_channels=192, expand=6, kernel_size=5, stride=1, dropout_rate=dropout),
                                    MBConvBlock(in_channels=192, out_channels=192, expand=6, kernel_size=5, stride=1, dropout_rate=dropout),
                                    MBConvBlock(in_channels=192, out_channels=192, expand=6, kernel_size=5, stride=1, dropout_rate=dropout),
                                   )
        
        # stage8
        self.stage8 = nn.Sequential(MBConvBlock(in_channels=192, out_channels=320, expand=6, kernel_size=3, stride=1, dropout_rate=dropout))
        
        # stage9
        self.last_channels = math.ceil(1280*width)
        self.stage9 = nn.Conv2d(in_channels=320, out_channels=self.last_channels, kernel_size=1)
        
        # result
        self.out_layer = nn.Linear(self.last_channels, num_classes)


    def forward(self, x):
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)
        x = self.stage6(x)
        x = self.stage7(x)
        x = self.stage8(x)
        x = self.stage9(x)
        x = F.adaptive_avg_pool2d(x, (1,1)).view(-1, self.last_channels)
        x = self.out_layer(x)

        return x



In [6]:
def efficientnet_b0(num_classes=10):
    return EfficientNet(num_classes=num_classes, width=1.0, depth=1.0, resolution=224, dropout=0.2)


In [7]:
if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    x = torch.randn(4, 3, 224, 224).to(device)
    model = efficientnet_b0().to(device)
    output = model(x)
    print('output size:', output.size())
    
    summary(model, input_size=(3, 224, 224))

output size: torch.Size([4, 10])
